In [ ]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *
clean_data_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/clean_data/"
output_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/gathering"

In [ ]:
zone_altimetriche = gpd.read_file(clean_data_folder + "zone_altimetriche_piemonte/zone_altimetriche_piemonte.shp")
mappa_forestale = gpd.read_file(clean_data_folder + "mappa_forestale/mappa_forestale.shp")
mappa_floreale = gpd.read_file(clean_data_folder + "copertura10_agg2016/copertura10_agg2016.shp")
formazioni_lineari = gpd.read_file(clean_data_folder + "formazioni_lineari_agg2016/formazioni_lineari_agg2016.shp")
arboricoltura = gpd.read_file(clean_data_folder + "arboricoltura_agg2016/arboricoltura_agg2016.shp")
strade_principali = gpd.read_file(clean_data_folder + "strade_principali_piemonte/strade_principali_piemonte.shp")

---

# Vegetazione

In [ ]:
display(mappa_forestale.info())
display(mappa_floreale.info())
display(formazioni_lineari.info())
display(arboricoltura.info())

In [ ]:
# mappa_forestale.drop(columns=["SOTTOTIPO", "VARIANTE"], inplace=True)
# mappa_floreale.drop(columns=["SOTTOTIPO", "VARIANTE"], inplace=True)
# arboricoltura.drop(columns=["SOTTOTIPO", "VARIANTE"], inplace=True)

In [ ]:
formazioni_lineari

In [ ]:
# formazioni_lineari.rename(columns={'idgeo_ln_f': 'IDGEO_PL_T', 'FORMAZIONE': 'CATEGORIA', 'COMP_SPEC': 'TIPO', 'TIPO_STRU': 'DESTINAZ', "LUNGH": "ESTENSIONE"}, inplace=True)

In [ ]:
vegetazione = pd.concat([arboricoltura, mappa_forestale, mappa_floreale], ignore_index=True)
vegetazione.head()

In [ ]:
plt.imshow(pd.crosstab(vegetazione['CATEGORIA'], vegetazione['CODICE']), cmap="Blues")
plt.show()
# CODICE refers exactly to CATEGORIA 
(~(pd.crosstab(vegetazione['TIPIFORE'], vegetazione['TIPO']) == 0)).sum(axis=1).all()
# the map TIPIFORE to TIPO is injective, not the converse. Keep TIPO since it has less classes and more interpretability

In [ ]:
vegetazione['PRIORITA'].unique()

In [ ]:
# vegetazione.drop(columns=["CODICE", 'TIPIFORE', 'COD_CLC'], inplace=True)

In [ ]:
# add "Aree per cui non si prevede priorita' d'intervento" to Filiare since they are planted on purpose 
formazioni_lineari['PRIORITA'] = "Aree per cui non si prevede priorita' d'intervento"
vegetazione['ESTENSIONE'] = vegetazione['geometry'].area

In [ ]:
vegetazione_piemonte = pd.concat([vegetazione, formazioni_lineari], ignore_index=True)
vegetazione_piemonte.isna().sum()

In [ ]:
import missingno

missingno.matrix(vegetazione_piemonte);
missingno.matrix(vegetazione);

In [ ]:
# vegetazione_piemonte['INTERVENTO'].fillna("Formazione lineare", inplace=True)

In [ ]:
display(vegetazione_piemonte['PRIORITA'].unique())
display(vegetazione_piemonte['DESTINAZ'].unique())
display(vegetazione_piemonte['ASSETTO'].unique())

In [ ]:
no_prior = vegetazione_piemonte[vegetazione_piemonte['PRIORITA'].isna()]
no_dest = vegetazione_piemonte[vegetazione_piemonte['DESTINAZ'].isna()]
no_asset = vegetazione_piemonte[vegetazione_piemonte['ASSETTO'].isna()]

In [ ]:
def get_neigh(target, gdf, col):
    light=False
    n=1
    while(light == False):
        buffer = target.buffer(100*n)
        # Find polygons that intersect this buffer
        neighbors = gdf[gdf.intersects(buffer.union_all()) & (~gdf[col].isna())]
        neighbors = neighbors[neighbors.index != target.index[0]]
        n+=1
        if neighbors.shape[0] > 0:
            light=True


    return neighbors[col].value_counts()

In [ ]:
from tqdm import tqdm 

lst = [no_prior, no_dest, no_asset]

# assign the most common value of the neighbors in range of 100m (or the nearest one if there are no neighbors)
for a, col in enumerate(["PRIORITA", "DESTINAZ", "ASSETTO"]):
    for i in tqdm(lst[a].index):
        c= get_neigh(lst[a].loc[[i]], vegetazione_piemonte, col).index[0]
        vegetazione_piemonte.loc[i, col] = c

In [ ]:
# substitute linestrings with thin polygons to uniformise geometry
vegetazione_piemonte['geometry'] = vegetazione_piemonte['geometry'].apply(lambda geom: geom.buffer(0.5) if geom.geom_type == 'LineString' else geom)

In [ ]:
vegetazione_piemonte.info()

In [ ]:
vegetazione_piemonte.to_file('/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/vegetazione')

---

# Altimetria e strade

In [ ]:
zone_altimetriche

In [ ]:
zone_altimetriche.drop(columns='foglio').to_file('/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/altimetria_per_circoscrizione')

---

In [ ]:
strade_principali

In [ ]:
strade_principali.describe()

In [ ]:
# remove univariate columns
strade_principali.drop(columns=['GP_REX', 'GP_RAV', 'GP_RRG', 'GP_RSE', 'GP_RSI', 'GP_RSY', 'gp_gripreg'], inplace=True)
strade_principali

In [ ]:
from shapely.geometry import MultiPolygon
# make a polygon of roads
def buffer_linestring(x):
    if x.geom_type == 'LineString':
        return x.buffer(3)
    elif x.geom_type == 'MultiLineString':
        buffers = [line.buffer(3) for line in x.geoms]
        return MultiPolygon(buffers)

strade_principali['geometry'] = strade_principali['geometry'].apply(lambda x: buffer_linestring(x))

strade_principali.geom_type.unique()

In [ ]:
strade_principali.to_file('/nfs/home/genovese/thesis-wildfire-genovese/data/gathering/strade_principali')

---